# import

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
#from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.contrib import rnn
import numpy as np
import tensorflow.contrib.slim as slim
from compute_mcc import *
#import scipy.io as sio
import math
import h5py
#from compute_mcc import compute_mcc,metrics,_fast_hist,label_accuracy_score
from hilbert import hilbertCurve
#from compute_IoU import compute_precision,bb_IoU
# sys.path.append('.')
import os,sys
from scipy import signal
import time
import skimage
import skimage.io, skimage.transform
from skimage.transform import resize
from skimage.util import view_as_windows
import scipy.misc
import scipy.io as sio
from skimage import img_as_uint
import matplotlib.pyplot as plt
# import pandas
import glob 
import datetime
from skimage.color import rgb2ycbcr 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pywt
# import cv2
import re
from PIL import Image
import os
import bisect
import pickle

# Fileters

In [3]:
Bayar_Kernel = np.array([
    [[0,0,0,0,0],[0,(1.0/4)*-1,(1.0/4)*2,(1.0/4)*-1,0],[0,(1.0/4)*2,(1.0/4)*-4,(1.0/4)*2,0],[0,(1.0/4)*-1,(1.0/4)*2,(1.0/4)*-1,0],[0,0,0,0,0]],
    [[(1.0/12)*-1,(1.0/12)*2,(1.0/12)*-2,(1.0/12)*2,(1.0/12)*-1],[(1.0/12)*2,(1.0/12)*-6,(1.0/12)*8,(1.0/12)*-6,(1.0/12)*2],[(1.0/12)*-2,(1.0/12)*8,(1.0/12)*-12,(1.0/12)*8,(1.0/12)*-2],[(1.0/12)*2,(1.0/12)*-6,(1.0/12)*8,(1.0/12)*-6,(1.0/12)*2],[(1.0/12)*-1,(1.0/12)*2,(1.0/12)*-2,(1.0/12)*2,(1.0/12)*-1]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,(1.0/2)*1,(1.0/2)*-2,(1.0/2)*1,0],[0,0,0,0,0],[0,0,0,0,0]]
])
Bayar_Kernel = np.vstack((Bayar_Kernel, Bayar_Kernel, Bayar_Kernel)).reshape(3, 3, 5, 5).transpose(2,3,0,1)

In [4]:
SRM_Kernel = np.array([
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,-1,1,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,1,-1,0,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,1,0,0],[0,0,-1,0,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,-1,0,0],[0,0,1,0,0],[0,0,0,0,0]],
])
SRM_Kernel = np.vstack((SRM_Kernel, SRM_Kernel, SRM_Kernel)).reshape(3, 4, 5, 5).transpose(2,3,0,1)

In [12]:
SRM_Kernel = np.array([
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,-1,1,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,1,-1,0,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,1,0,0],[0,0,-1,0,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,-1,0,0],[0,0,1,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,1,0],[0,0,-1,0,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,-1,0,0],[0,0,0,1,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,1,0,0,0],[0,0,-1,0,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,-1,0,0],[0,1,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,1,-2,1,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,1,0,0],[0,0,-2,0,0],[0,0,1,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,1,0,0,0],[0,0,-2,0,0],[0,0,0,1,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,1,0],[0,0,-2,0,0],[0,1,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,1,-3,3,-1],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[-1,3,-3,1,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,-1,0,0],[0,0,3,0,0],[0,0,-3,0,0],[0,0,1,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,1,0,0],[0,0,-3,0,0],[0,0,3,0,0],[0,0,-1,0,0]],
    [[0,0,0,0,-1],[0,0,0,3,0],[0,0,-3,0,0],[0,1,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,1,0,0,0],[0,0,-3,0,0],[0,0,0,3,0],[0,0,0,0,-1]],
    [[-1,0,0,0,0],[0,3,0,0,0],[0,0,-3,0,0],[0,0,0,1,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,1,0],[0,0,-3,0,0],[0,3,0,0,0],[-1,0,0,0,0]],
    [[0,0,0,0,0],[0,-1,2,-1,0],[0,2,-4,2,0],[0,-1,2,-1,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,2,-1,0],[0,0,-4,2,0],[0,0,2,-1,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,-1,2,0,0],[0,2,-4,0,0],[0,-1,2,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,-1,2,-1,0],[0,2,-4,2,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,2,-4,2,0],[0,-1,2,-1,0],[0,0,0,0,0]],
    [[-1,2,-2,2,-1],[2,-6,8,-6,2],[-2,8,-12,8,-2],[2,-6,8,-6,2],[-1,2,-2,2,-1]],
    [[0,0,-2,2,-1],[0,0,8,-6,2],[0,0,-12,8,-2],[0,0,8,-6,2],[0,0,-2,2,-1]],
    [[-1,2,-2,0,0],[2,-6,8,0,0],[-2,8,-12,0,0],[2,-6,8,0,0],[-1,2,-2,0,0]],
    [[-1,2,-2,2,-1],[2,-6,8,-6,2],[-2,8,-12,8,-2],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[-2,8,-12,8,-2],[2,-6,8,-6,2],[-1,2,-2,2,-1]]
])
SRM_Kernels = np.vstack((SRM_Kernel, SRM_Kernel, SRM_Kernel)).reshape(3, 30, 5, 5).transpose(2,3,0,1)

In [84]:
tf.reset_default_graph()
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
log_device_placement = True
# Parameters
lr = 0.00003
training_iters = 50000000
batch_size = 30
display_step = 10
nb_nontamp_img=16960
nb_tamp_img=68355
nbFilter=32


# LSTM network parameters
n_input = 240 # data input (img shape: 64x64)
n_steps = 64 # timesteps
nBlock=int(math.sqrt(n_steps))
n_hidden = 64# hidden layer num of features
nStride=int(math.sqrt(n_hidden))
# other parameters
imSize=256
# Network Parameters
n_classes = 2 # manipulated vs unmanipulated
mx=127.0

# tf Graph input
input_layer = tf.placeholder("float", [None, imSize,imSize,3])
y= tf.placeholder("float", [2,None, imSize,imSize])
freqFeat=tf.placeholder("float", [None, 248,248,3])
# freqFeat=tf.placeholder("float", [None, 64,240])
# freqFeat=tf.placeholder("float", [None, 256,256,3])
filter = tf.Variable(tf.random_normal([5,5,3,9]))
ratio=15.0 #tf.placeholder("float",[1])
#out_rnn=tf.placeholder("float", [None, 128,128,3])
# W1 = tf.get_variable('W1', [5,5,3, 10], tf.float32, xavier_initializer())
# b1 = tf.Variable(tf.random_normal([5,5,3]))

############################################################################
#total_layers = 25 #Specify how deep we want our network
units_between_stride = 2
upsample_factor=16
beta=.01
outSize=16
############################################################################
seq = np.linspace(0,63,64).astype(int)
order3 = hilbertCurve(3)
order3 = np.reshape(order3,(64))
print(seq)
print(order3)
hilbert_ind = np.lexsort((seq,order3))
actual_ind=np.lexsort((seq,hilbert_ind))

weights = {
    'out': tf.Variable(tf.random_normal([64,64,nbFilter]))
}
biases = {
    'out': tf.Variable(tf.random_normal([nbFilter]))
}

atrous_fil = tf.Variable(tf.random_normal([3, 3, 256,256]),name = 'atrous_fil')
atrous_fil_1 =tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil_1')
atrous_fil_2 =tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil_2')


atrous_fil1 = tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil1')
atrous_fil2 = tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil2')
atrous_fil3 = tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil3')

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
[ 0  1 14 15 16 19 20 21  3  2 13 12 17 18 23 22  4  7  8 11 30 29 24 25
  5  6  9 10 31 28 27 26 58 57 54 53 32 35 36 37 59 56 55 52 33 34 39 38
 60 61 50 51 46 45 40 41 63 62 49 48 47 44 43 42]


In [20]:
tf.reset_default_graph()
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
log_device_placement = True
lr = 0.00003
training_iters = 50000000
batch_size = 30
display_step = 10
nb_nontamp_img=16960
nb_tamp_img=68355
nbFilter=32


# LSTM network parameters
n_input = 240 # data input (img shape: 64x64)
n_steps = 64 # timesteps
nBlock=int(math.sqrt(n_steps))
n_hidden = 64# hidden layer num of features
nStride=int(math.sqrt(n_hidden))
# other parameters
imSize=256
# Network Parameters
n_classes = 2 # manipulated vs unmanipulated


# tf Graph input
input_layer = tf.placeholder("float", [None, imSize,imSize,3])
y= tf.placeholder("float", [2,None, imSize,imSize])
freqFeat=tf.placeholder("float", [None, 64,240])
ratio=15.0 #tf.placeholder("float",[1])
#out_rnn=tf.placeholder("float", [None, 128,128,3])



############################################################################
#total_layers = 25 #Specify how deep we want our network
units_between_stride = 2
upsample_factor=16
n_classes=2
beta=.01
outSize=16
############################################################################
seq = np.linspace(0,63,64).astype(int)
order3 = hilbertCurve(3)
order3 = np.reshape(order3,(64))
hilbert_ind = np.lexsort((seq,order3))
actual_ind=np.lexsort((seq,hilbert_ind))

weights = {
    'out': tf.Variable(tf.random_normal([64,64,nbFilter]))
}
biases = {
    'out': tf.Variable(tf.random_normal([nbFilter]))
}

## base-model 

In [21]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        # layer1: resblock, input size(256,256)
        layer1 = slim.conv2d(input_layer,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(0))
        layer1 =resUnit(layer1,1,nbFilter)
        layer1 = tf.nn.relu(layer1)
        layer2=slim.max_pool2d(layer1, [2, 2], scope='pool_'+str(1))
        # layer2: resblock, input size(128,128)   
        layer2 = slim.conv2d(layer2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        layer2 =resUnit(layer2,2,2*nbFilter)
        layer2 = tf.nn.relu(layer2)
        layer3=slim.max_pool2d(layer2, [2, 2], scope='pool_'+str(2))
        # layer3: resblock, input size(64,64) 
        layer3 = slim.conv2d(layer3,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        layer3 =resUnit(layer3,3,4*nbFilter)
        layer3 = tf.nn.relu(layer3)
        layer4=slim.max_pool2d(layer3, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        layer4 = slim.conv2d(layer4,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        layer4 =resUnit(layer4,4,8*nbFilter)
        layer4 = tf.nn.relu(layer4)
        layer4=slim.max_pool2d(layer4, [2, 2], scope='pool_'+str(4))
        # end of layer4: resblock, input size(16,16)

        # lstm network 
        patches=tf.transpose(freqFeat,[1,0,2])
        patches=tf.gather(patches,hilbert_ind)
        patches=tf.transpose(patches,[1,0,2])         
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        print('lstm_out3:',lstm_out.shape)
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        print('lstm_out3:',lstm_out.shape)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])

        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer4,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        print('joint_out:',joint_out.shape)
        
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        print('upsampled_layer4:',upsampled_layer4.shape)
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(4))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        print('upsampled_layer4:',upsampled_layer4.shape)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        print('upsampled_layer5:',upsampled_layer5.shape)
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)


    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


lstm_out3: (64, ?, 32)
lstm_out3: (30, 8, 8, 32)
joint_out: (30, 16, 16, 64)
upsampled_layer4: (30, 64, 64, 64)
upsampled_layer4: (30, 64, 64, 2)
upsampled_layer5: (30, 256, 256, 2)


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


## VGG_AR_1

In [99]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, Bayar_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        
        # layer4: resblock, input size(32,32) 
#         print('Pool_4:',Pool_4.shape)
        
        Conv_10_AC = slim.conv2d(ReLu_9,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(10))
        Tan_1 = tf.nn.relu(Conv_10_AC)
        
        Conv_11_AC = tf.nn.atrous_conv2d(Tan_1,atrous_fil,rate =3,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.tanh(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =5,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.tanh(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =7,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.tanh(BN_4)
        output = ReLu_9 + Tan_4
#         Pool_4 = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
        
#         layer55 = tf.nn.relu(output)
       
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(13))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(14))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(15))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(16))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
        layer12 = tf.nn.relu(layer10)
        layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
        print('layer13:',layer13.shape)
     
        # lstm network 
        layer55 = slim.conv2d(freqFeat,16,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(17))
        layer55 = tf.nn.relu(layer55)
        layer66 = slim.conv2d(layer55,1,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(18))
        layer66 = tf.nn.relu(layer66)
        print('layer6_shape:',layer66.shape)  # ( bs, 256, 256, 1)
        
        layer77=tf.transpose(layer66,[0,3,1,2])
        print('layer7_shape:',layer77.shape)  # ( bs, 256, 256, 1)
        
        y_list = tf.split(layer77,8,axis=3)
        print('y_list_shape2:',len(y_list))
        xy_list = [tf.split(x,8,axis =2) for x in y_list] ##
        print('xy_list_shape2:',len(xy_list))
        xy = [item for items in xy_list for item in items]
        print('xy_shape:',len(xy))
#         xy = torch.cat(xy,1)
        xy = tf.concat(xy,1)
        print('xy_shape2:',xy.shape)
#         patches = xy.view(-1,64,64)#
        patches = tf.reshape(xy,(-1,64,31*31))
        print('patches_shape2:',patches.shape)
        
        
#         patches=tf.transpose(freqFeat,[1,0,2])
#         patches=tf.gather(patches,hilbert_ind)
#         patches=tf.transpose(patches,[1,0,2])
#         print('patches:',patches.shape)
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
        print('lstm_out:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer13,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)


    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


('concat:', TensorShape([Dimension(None), Dimension(256), Dimension(256), Dimension(13)]))
('Pool_1:', TensorShape([Dimension(None), Dimension(128), Dimension(128), Dimension(32)]))
('Pool_2:', TensorShape([Dimension(None), Dimension(64), Dimension(64), Dimension(64)]))
('Pool_3:', TensorShape([Dimension(None), Dimension(32), Dimension(32), Dimension(128)]))
('layer10:', TensorShape([Dimension(None), Dimension(32), Dimension(32), Dimension(256)]))
('layer13:', TensorShape([Dimension(None), Dimension(16), Dimension(16), Dimension(256)]))
('layer6_shape:', TensorShape([Dimension(None), Dimension(248), Dimension(248), Dimension(1)]))
('layer7_shape:', TensorShape([Dimension(None), Dimension(1), Dimension(248), Dimension(248)]))
('y_list_shape2:', 8)
('xy_list_shape2:', 8)
('xy_shape:', 64)
('xy_shape2:', TensorShape([Dimension(None), Dimension(64), Dimension(31), Dimension(31)]))
('patches_shape2:', TensorShape([Dimension(None), Dimension(64), Dimension(961)]))
('lstm_out:', TensorShape([

## VGG_AR2

In [ ]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, Bayar_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        layer3 = tf.nn.conv2d(input_layer, SRM_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2,layer3], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        
        # layer4: resblock, input size(32,32) 
#         print('Pool_4:',Pool_4.shape)
        
        Conv_10_AC = slim.conv2d(ReLu_9,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(10))
        Tan_1 = tf.nn.relu(Conv_10_AC)
        
        Conv_11_AC = tf.nn.atrous_conv2d(Tan_1,atrous_fil,rate =3,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.relu(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =5,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.relu(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =7,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.relu(BN_4)
        output = ReLu_9 + Tan_4
#         output = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
        
#         layer55 = tf.nn.relu(output)
       
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(14))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(15))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(16))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(17))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
        layer12 = tf.nn.relu(layer10)
        layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
        print('layer13:',layer13.shape)
     
        # lstm network 
        layer55 = slim.conv2d(freqFeat,16,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(18))
        layer55 = tf.nn.relu(layer55)
        layer66 = slim.conv2d(layer55,1,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(19))
        layer66 = tf.nn.relu(layer66)
        print('layer6_shape:',layer66.shape)  # ( bs, 256, 256, 1)
        
        layer77=tf.transpose(layer66,[0,3,1,2])
        print('layer7_shape:',layer77.shape)  # ( bs, 256, 256, 1)
        
        y_list = tf.split(layer77,8,axis=3)
        print('y_list_shape2:',len(y_list))
        xy_list = [tf.split(x,8,axis =2) for x in y_list] ##
        print('xy_list_shape2:',len(xy_list))
        xy = [item for items in xy_list for item in items]
        print('xy_shape:',len(xy))
#         xy = torch.cat(xy,1)
        xy = tf.concat(xy,1)
        print('xy_shape2:',xy.shape)
#         patches = xy.view(-1,64,64)#
        patches = tf.reshape(xy,(-1,64,31*31))
        print('patches_shape2:',patches.shape)
        
        
#         patches=tf.transpose(freqFeat,[1,0,2])
#         patches=tf.gather(patches,hilbert_ind)
#         patches=tf.transpose(patches,[1,0,2])
#         print('patches:',patches.shape)
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
        print('lstm_out:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer13,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)


    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


### VGG_AR_3

In [15]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, Bayar_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        layer3 = tf.nn.conv2d(input_layer, SRM_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2,layer3], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        
        # layer4: resblock, input size(32,32) 
#         print('Pool_4:',Pool_4.shape)
        
        Conv_10_AC = slim.conv2d(ReLu_9,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(10))
        Tan_1 = tf.nn.relu(Conv_10_AC)
        
        Conv_11_AC = tf.nn.atrous_conv2d(Tan_1,atrous_fil,rate =3,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.relu(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =5,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.relu(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =7,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.relu(BN_4)
        output = ReLu_9 + Tan_4
        output = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
        
#         layer55 = tf.nn.relu(output)
       
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(14))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(15))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(16))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(17))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
#         layer12 = tf.nn.relu(layer10)
#         layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
#         print('layer13:',layer13.shape)
     
        # lstm network 
        layer55 = slim.conv2d(freqFeat,16,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(18))
        layer55 = tf.nn.relu(layer55)
        layer66 = slim.conv2d(layer55,1,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(19))
        layer66 = tf.nn.relu(layer66)
        print('layer6_shape:',layer66.shape)  # ( bs, 256, 256, 1)
        
        layer77=tf.transpose(layer66,[0,3,1,2])
        print('layer7_shape:',layer77.shape)  # ( bs, 256, 256, 1)
        
        y_list = tf.split(layer77,8,axis=3)
        print('y_list_shape2:',len(y_list))
        xy_list = [tf.split(x,8,axis =2) for x in y_list] ##
        print('xy_list_shape2:',len(xy_list))
        xy = [item for items in xy_list for item in items]
        print('xy_shape:',len(xy))
#         xy = torch.cat(xy,1)
        xy = tf.concat(xy,1)
        print('xy_shape2:',xy.shape)
#         patches = xy.view(-1,64,64)#
        patches = tf.reshape(xy,(-1,64,31*31))
        print('patches_shape2:',patches.shape)
        
        
#         patches=tf.transpose(freqFeat,[1,0,2])
#         patches=tf.gather(patches,hilbert_ind)
#         patches=tf.transpose(patches,[1,0,2])
#         print('patches:',patches.shape)
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
        print('lstm_out:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer10,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)


    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


concat: (?, 256, 256, 16)
Pool_1: (?, 128, 128, 32)
Pool_2: (?, 64, 64, 64)
Pool_3: (?, 32, 32, 128)
layer10: (?, 16, 16, 256)
layer6_shape: (?, 248, 248, 1)
layer7_shape: (?, 1, 248, 248)
y_list_shape2: 8
xy_list_shape2: 8
xy_shape: 64
xy_shape2: (?, 64, 31, 31)
patches_shape2: (?, 64, 961)
lstm_out: (30, 16, 16, 32)
top: (?, 16, 16, 32)


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### VGG_AR_4 _LSTM

In [24]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, Bayar_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        layer3 = tf.nn.conv2d(input_layer, SRM_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2,layer3], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        
        # layer4: resblock, input size(32,32) 
#         print('Pool_4:',Pool_4.shape)
        
        Conv_10_AC = slim.conv2d(ReLu_9,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(10))
        Tan_1 = tf.nn.relu(Conv_10_AC)
        
        Conv_11_AC = tf.nn.atrous_conv2d(Tan_1,atrous_fil,rate =3,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.relu(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =5,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.relu(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =7,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.relu(BN_4)
        output = ReLu_9 + Tan_4
#         output = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
        
#         layer55 = tf.nn.relu(output)
       
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(14))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(15))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(16))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(17))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
        layer12 = tf.nn.relu(layer10)
        layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
        print('layer13:',layer13.shape)
     
        # lstm network 
        layer55 = slim.conv2d(freqFeat,16,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(18))
        layer55 = tf.nn.relu(layer55)
        layer66 = slim.conv2d(layer55,1,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(19))
        layer66 = tf.nn.relu(layer66)
        print('layer6_shape:',layer66.shape)  # ( bs, 256, 256, 1)
        
        layer77=tf.transpose(layer66,[0,3,1,2])
        print('layer7_shape:',layer77.shape)  # ( bs, 256, 256, 1)
        
        y_list = tf.split(layer77,8,axis=3)
        print('y_list_shape2:',len(y_list))
        xy_list = [tf.split(x,8,axis =2) for x in y_list] ##
        print('xy_list_shape2:',len(xy_list))
        xy = [item for items in xy_list for item in items]
        print('xy_shape:',len(xy))
#         xy = torch.cat(xy,1)
        xy = tf.concat(xy,1)
        print('xy_shape2:',xy.shape)
#         patches = xy.view(-1,64,64)#
        patches = tf.reshape(xy,(-1,64,32*32))
        print('patches_shape2:',patches.shape)
        
        
#         patches=tf.transpose(freqFeat,[1,0,2])
#         patches=tf.gather(patches,hilbert_ind)
#         patches=tf.transpose(patches,[1,0,2])
#         print('patches:',patches.shape)
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
        print('lstm_out:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer13,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)


    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


concat: (?, 256, 256, 16)
Pool_1: (?, 128, 128, 32)
Pool_2: (?, 64, 64, 64)
Pool_3: (?, 32, 32, 128)
layer10: (?, 32, 32, 256)
layer13: (?, 16, 16, 256)
layer6_shape: (?, 256, 256, 1)
layer7_shape: (?, 1, 256, 256)
y_list_shape2: 8
xy_list_shape2: 8
xy_shape: 64
xy_shape2: (?, 64, 32, 32)
patches_shape2: (?, 64, 1024)
lstm_out: (30, 16, 16, 32)
top: (?, 16, 16, 32)


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


## PL-GNet

In [85]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, Bayar_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        layer3 = tf.nn.conv2d(input_layer, SRM_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2,layer3], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        print('ReLu_9:',ReLu_9.shape)
        # layer4: resblock, input size(32,32) 
#         print('Pool_4:',Pool_4.shape)
        
#         Conv_10_AC = slim.conv2d(ReLu_9,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(10))
#         Tan_1 = tf.nn.relu(Conv_10_AC)
        
        Conv_11_AC = tf.nn.atrous_conv2d(ReLu_9,atrous_fil,rate =2,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.relu(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =2,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.relu(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =2,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.relu(BN_4)
        output = ReLu_9 + Tan_4
#         output = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
        
#         layer55 = tf.nn.relu(output)
       
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(14))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(15))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(16))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(17))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
        layer12 = tf.nn.relu(layer10)
        layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
        print('layer13:',layer13.shape)
     
        # lstm network 
        layer55 = slim.conv2d(freqFeat,16,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(18))
        layer55 = tf.nn.relu(layer55)
        layer66 = slim.conv2d(layer55,1,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(19))
        layer66 = tf.nn.relu(layer66)
        print('layer6_shape:',layer66.shape)  # ( bs, 256, 256, 1)
        
        layer77=tf.transpose(layer66,[0,3,1,2])
        print('layer7_shape:',layer77.shape)  # ( bs, 256, 256, 1)
        
        y_list = tf.split(layer77,8,axis=3)
        print('y_list_shape2:',len(y_list))
        xy_list = [tf.split(x,8,axis =2) for x in y_list] ##
        print('xy_list_shape2:',len(xy_list))
        xy = [item for items in xy_list for item in items]
        print('xy_shape:',len(xy))
#         xy = torch.cat(xy,1)
        xy = tf.concat(xy,1)
        print('xy_shape2:',xy.shape)
#         patches = xy.view(-1,64,64)#
        patches = tf.reshape(xy,(-1,64,31*31))
        print('patches_shape2:',patches.shape)
        
        
#         patches=tf.transpose(freqFeat,[1,0,2])
#         patches=tf.gather(patches,hilbert_ind)
#         patches=tf.transpose(patches,[1,0,2])
#         print('patches:',patches.shape)
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        print('lstm_out1:',lstm_out.shape)
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        print('lstm_out2:',lstm_out.shape)
        
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        print('lstm_out3:',lstm_out.shape)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
        print('lstm_out4:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer13,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        print('joint_out:',joint_out.shape)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)
    print('upsampled_logits_shape:',upsampled_logits.shape)

    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    print('flat_pred_shape:',flat_pred.shape)
    
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


concat: (?, 256, 256, 16)
Pool_1: (?, 128, 128, 32)
Pool_2: (?, 64, 64, 64)
Pool_3: (?, 32, 32, 128)
ReLu_9: (?, 32, 32, 256)
layer10: (?, 32, 32, 256)
layer13: (?, 16, 16, 256)
layer6_shape: (?, 248, 248, 1)
layer7_shape: (?, 1, 248, 248)
y_list_shape2: 8
xy_list_shape2: 8
xy_shape: 64
xy_shape2: (?, 64, 31, 31)
patches_shape2: (?, 64, 961)
lstm_out1: (64, ?, 32)
lstm_out2: (?, 64, 32)
lstm_out3: (30, 8, 8, 32)
lstm_out4: (30, 16, 16, 32)
top: (?, 16, 16, 32)
joint_out: (30, 16, 16, 64)
upsampled_logits_shape: (30, 256, 256, 2)
flat_pred_shape: (1966080, 2)


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


### VGGAR_no_LSTM

In [5]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, Bayar_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        layer3 = tf.nn.conv2d(input_layer, SRM_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2,layer3], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        print('ReLu_9:',ReLu_9.shape)
        # layer4: resblock, input size(32,32) 
#         print('Pool_4:',Pool_4.shape)
        
#         Conv_10_AC = slim.conv2d(ReLu_9,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(10))
#         Tan_1 = tf.nn.relu(Conv_10_AC)
        
        Conv_11_AC = tf.nn.atrous_conv2d(ReLu_9,atrous_fil,rate =2,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.relu(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =2,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.relu(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =2,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.relu(BN_4)
        output = ReLu_9 + Tan_4
#         output = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
        
#         layer55 = tf.nn.relu(output)
       
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(14))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(15))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(16))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(17))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
        layer12 = tf.nn.relu(layer10)
        layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
        print('layer13:',layer13.shape)
     
#         # lstm network 
#         layer55 = slim.conv2d(freqFeat,16,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(18))
#         layer55 = tf.nn.relu(layer55)
#         layer66 = slim.conv2d(layer55,1,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(19))
#         layer66 = tf.nn.relu(layer66)
#         print('layer6_shape:',layer66.shape)  # ( bs, 256, 256, 1)
        
#         layer77=tf.transpose(layer66,[0,3,1,2])
#         print('layer7_shape:',layer77.shape)  # ( bs, 256, 256, 1)
        
#         y_list = tf.split(layer77,8,axis=3)
#         print('y_list_shape2:',len(y_list))
#         xy_list = [tf.split(x,8,axis =2) for x in y_list] ##
#         print('xy_list_shape2:',len(xy_list))
#         xy = [item for items in xy_list for item in items]
#         print('xy_shape:',len(xy))
# #         xy = torch.cat(xy,1)
#         xy = tf.concat(xy,1)
#         print('xy_shape2:',xy.shape)
# #         patches = xy.view(-1,64,64)#
#         patches = tf.reshape(xy,(-1,64,31*31))
#         print('patches_shape2:',patches.shape)
        
        
# #         patches=tf.transpose(freqFeat,[1,0,2])
# #         patches=tf.gather(patches,hilbert_ind)
# #         patches=tf.transpose(patches,[1,0,2])
# #         print('patches:',patches.shape)
#         # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
#         xCell=tf.unstack(patches, n_steps, 1)
#         # 2 stacked layers
#         stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
#         out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
#         # organizing the lstm output
#         out=tf.gather(out,actual_ind)
#         # convert to lstm output (64,batchSize,nbFilter)
#         lstm_out=tf.matmul(out,weights['out'])+biases['out']
#         print('lstm_out1:',lstm_out.shape)
#         lstm_out=tf.transpose(lstm_out,[1,0,2])
#         print('lstm_out2:',lstm_out.shape)
        
#         # convert to size(batchSize, 8,8, nbFilter)
#         lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
#         # perform batch normalization and activiation
#         lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
#         lstm_out=tf.nn.relu(lstm_out)
#         print('lstm_out3:',lstm_out.shape)
#         # upsample lstm output to (batchSize, 16,16, nbFilter)
#         temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
#         uShape1=tf.shape(temp)
#         upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
#         upsample_filter_tensor = tf.constant(upsample_filter_np)
#         lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
#         print('lstm_out4:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer13,2*nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
#         joint_out=tf.concat([top,lstm_out],3)
#         print('joint_out:',joint_out.shape)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(top, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)
    print('upsampled_logits_shape:',upsampled_logits.shape)

    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    print('flat_pred_shape:',flat_pred.shape)
    
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


concat: (?, 256, 256, 16)
Pool_1: (?, 128, 128, 32)
Pool_2: (?, 64, 64, 64)
Pool_3: (?, 32, 32, 128)
ReLu_9: (?, 32, 32, 256)
layer10: (?, 32, 32, 256)
layer13: (?, 16, 16, 256)
top: (?, 16, 16, 64)
upsampled_logits_shape: (30, 256, 256, 2)
flat_pred_shape: (1966080, 2)


### VGG_AR+original-feat

In [32]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, Bayar_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
#         layer3 = tf.nn.conv2d(input_layer, SRM_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        
        # layer4: resblock, input size(32,32) 
#         print('Pool_4:',Pool_4.shape)
        
#         Conv_10_AC = slim.conv2d(ReLu_9,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(10))
#         Tan_1 = tf.nn.relu(Conv_10_AC)
        
        Conv_11_AC = tf.nn.atrous_conv2d(ReLu_9,atrous_fil,rate =2,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.relu(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =2,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.relu(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =2,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.relu(BN_4)
        output = ReLu_9 + Tan_4
#         output = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
        
#         layer55 = tf.nn.relu(output)
       
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(14))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(15))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(16))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(17))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
        layer12 = tf.nn.relu(layer10)
        layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
        print('layer13:',layer13.shape)
     
        # lstm network 
        
         # lstm network 
        patches=tf.transpose(freqFeat,[1,0,2])
        patches=tf.gather(patches,hilbert_ind)
        patches=tf.transpose(patches,[1,0,2])         
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
        print('lstm_out:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer13,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)


    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


concat: (?, 256, 256, 12)
Pool_1: (?, 128, 128, 32)
Pool_2: (?, 64, 64, 64)
Pool_3: (?, 32, 32, 128)
layer10: (?, 32, 32, 256)
layer13: (?, 16, 16, 256)
lstm_out: (30, 16, 16, 32)
top: (?, 16, 16, 32)


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [67]:
from numpy import *
a = np.arange(12).reshape(2,3,2)
b = np.arange(24).reshape(2,3,4)

c = concatenate((a,b),axis=2)

print(c.shape)

(2, 3, 6)


# load data

In [13]:
# loading Synthesized data
# freq1 = np.load('../dataset_npy/synthesized_imgs/Synthesized_imgS_feat.npy')

Img = np.load('../dataset_npy/synthesized_imgs/Synthesized_imgS.npy')
print('Img.....OK')
Lab = np.load('../dataset_npy/synthesized_imgs/Synthesized_labelS.npy')
print('Lab....OK')
freq1 = np.load('../dataset_npy/synthesized_imgs/Synthesized_imgS_cohere_feat.npy')
print('freq1...OK')

# loading IEEE IFS data

#######----train
# freq2 = np.load('../dataset_npy/IFS/train/IFS_train_img_feat.npy')
# print ('freq2.shape=',np.shape(freq2))
# nI = np.load('../dataset_npy/IFS/train/IFS_train_img.npy')
# print ('nI.shape=',np.shape(nI))
# nL = np.load('../dataset_npy/IFS/train/IFS_train_label.npy')
# print ('nL.shape=',np.shape(nL))

# # validation set
# dx=nI[-batch_size:]
# #dx=np.float32(dx)
# dx1=freq2[-batch_size:]
# dy=nL[-batch_size:]
# # separate out the training set from val set
# nI=nI[:-batch_size]
# nL=nL[:-batch_size]
# freq2 = freq2[:-batch_size]
# print ('nI.shape=',np.shape(nI))
# print ('dx.shape=',np.shape(dx)) 

#####------test
# freq3 = np.load('../dataset_npy/IFS/test/IFS_test_img_feat.npy')
# print ('freq3.shape=',np.shape(freq3))
# nc17_img = np.load('../dataset_npy/IFS/test/IFS_test_img.npy')
# nb_nc17_img=np.shape(nc17_img)[0]
# print ('nc17_img.shape=',np.shape(nc17_img))
# nc17_lab = np.load('../dataset_npy/IFS/test/IFS_test_label.npy')
# print ('nc17_lab.shape=',np.shape(nc17_lab))


# loading NC_16 data

Img_NC16 = np.load('../dataset_npy/NC_16/train/NC16_train_img.npy')
Img_NC16_img=np.shape(Img_NC16)
print ('Img_NC16_img.shape=',Img_NC16_img)
Lab_NC16 = np.load('../dataset_npy/NC_16/train/NC16_train_label.npy')
# feat_nc16 = np.load('../dataset_npy/NC_16/train/NC16_train_img_feat.npy')
feat_nc16 = np.load('../dataset_npy/NC_16/train/NC16_train_img_cohere_feat.npy')

vx=Img_NC16[-40:]
vx1=feat_nc16[-40:]
vy=Lab_NC16[-40:]


Img_NC16 = Img_NC16[:-40]
Lab_NC16 = Lab_NC16[:-40]
feat_nc16 = feat_nc16[:-40]
print('Img_NC16.shape=',Img_NC16.shape)

tx = np.load('../dataset_npy/NC_16/test/NC16_test_img.npy')
ty = np.load('../dataset_npy/NC_16/test/NC16_test_label.npy')
# freq4 = np.load('../dataset_npy/NC_16/test/NC16_test_img_feat.npy')
freq4 = np.load('../dataset_npy/NC_16/test/NC16_test_img_cohere_feat.npy')
print('freq4.shape=',freq4.shape)

Img.....OK
Lab....OK
freq1...OK
Img_NC16_img.shape= (798, 256, 256, 3)
Img_NC16.shape= (758, 256, 256, 3)
freq4.shape= (266, 248, 248, 3)


In [6]:
Img = np.load('../dataset_npy/synthesized_imgs/Synthesized_imgS.npy')
print('Img.....OK')
Lab = np.load('../dataset_npy/synthesized_imgs/Synthesized_labelS.npy')
print('Lab....OK')
freq1 = np.load('../dataset_npy/synthesized_imgs/Synthesized_imgS_cohere_feat.npy')
print('freq1...OK')
print('img_shape:',Img.shape)

Img.....OK
Lab....OK
freq1...OK
img_shape: (67179, 256, 256, 3)


## function

In [7]:
def next_batch(size, batch_size, batch_ratio=1.0):
    import random
    rand = list(range(size))
    while True:
        random.shuffle(rand)
        for i in range(int(batch_ratio * size // batch_size)):
            yield rand[i*batch_size:(i+1)*batch_size]

In [8]:
def compute_pos_neg(y_actual, y_hat):
    TP = 0; FP = 0;TN = 0; FN = 0
#     print('y_hat_shape',y_hat.shape)
#     print('y_actual_shape',y_actual.shape)
#     print('y_hat',y_hat)
#     print('y_actual',y_actual)
    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==0:
            TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN += 1

    return TP,FP,TN,FN

In [9]:
def metrics(TP,FP,TN,FN):
    a=TP+FP
    b=TP+FN
    c=TN+FP
    d=TN+FN
    mcc=((TP*TN)-(FP*FN))/(math.sqrt(float(a*b*c*d)+0.0001))
    F1=(2*TP)/float(2*TP+FP+FN+.0000001)
    precision=TP/float(TP+FP+.0000001)
    #recall=TP/float(TP+FN+.0000001)
    return precision


In [17]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 100000
batch_size = 32
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tnb = next_batch(Lab.shape[0], batch_size)
    vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
        print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
            
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            
            learn_step +=1  
        train_loss /= train_batches
        vali_loss = 0
        vali_accuracy = 0
        vali_batches = vx.shape[0]//batch_size
        print()
        
        print('vali_batches=',vali_batches)
        for j in range(vali_batches):
#             print('\rj: %d'%j)
            select = next(vnb)
            batch_X = vx[select]
            batch_Y  = vy[select]
            batch_X1 = vx1[select]
            batch_X=np.multiply(batch_X,1.0/mx)
            rev_batch_y=np.array(conv_mask_gt(batch_Y))

            vali_loss += sess.run(loss, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
            vali_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
        vali_loss /= vali_batches
        vali_accuracy /= vali_batches
        
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        print()
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TLoss:%.6f VLoss:%f VAcc:%f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, vali_loss, vali_accuracy))
        if learn_step > train_step:
                break
        print('learn_step=',learn_step)
        if learn_step % int(show_freq * train_batches)==0:
            
            
            TP = 0; FP = 0;TN = 0; FN = 0 
            #TP1=0;FP1=0
            num_images=batch_size
            n_chunks=np.shape(tx)[0]//batch_size
            tAcc=np.zeros(n_chunks)

            for chunk in range(0,n_chunks):               
                tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
                tx1_batch=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=conv_mask_gt(ty_batch)
                tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch, freqFeat: tx1_batch})
                a,b,c,d=compute_pos_neg(y2,p2)

                TP+=a; FP+=b;TN+=c; FN+=d
                
            print("TP =",TP)
            print("FP = ",FP)
            print("TN = ",TN)
            print("FN = ",FN)
            
            prec=metrics(TP,FP,TN,FN)            
            test_accuracy=np.mean(tAcc)
            print("prec = ",prec)
            print("test_accuracy = ",test_accuracy)
           
            if prec > 0.45 :
                best_prec = prec
                save_path=saver.save(sess,'../model_shi/final_model_nist.ckpt')
                print ("Best Model Found on NC16...")
                print ( "prec = "+str(prec) + ", acc = "+ str(test_accuracy))

            saver.save(sess, '../model_shi/modelS%d.ckpt' % learn_step)
            
        
    saver.save(sess, '../model_shi/modelS.ckpt')

('total_epoch=', 23.0)
('train_batches=', 4198)
4197()
('vali_batches=', 2)
()
05-17 22:25:27 | Step:  4198(  1/ 23) LR:0.000030 TLoss:0.701346 VLoss:0.743232 VAcc:0.545825
('learn_step=', 4198)
('TP =', 610135)
('FP = ', 8702156)
('TN = ', 6904691)
('FN = ', 560234)
('prec = ', 0.065519322796076)
('test_accuracy = ', 0.4479186534881592)
('train_batches=', 4198)
4197()
('vali_batches=', 2)
()
05-17 22:42:34 | Step:  8396(  2/ 23) LR:0.000030 TLoss:0.691650 VLoss:0.736005 VAcc:0.552769
('learn_step=', 8396)
('TP =', 618166)
('FP = ', 8292456)
('TN = ', 7314391)
('FN = ', 552203)
('prec = ', 0.06937405716458324)
('test_accuracy = ', 0.47281742095947266)
('train_batches=', 4198)
4197()
('vali_batches=', 2)
()
05-17 22:59:29 | Step: 12594(  3/ 23) LR:0.000030 TLoss:0.690206 VLoss:0.732762 VAcc:0.557689
('learn_step=', 12594)
('TP =', 602830)
('FP = ', 8391745)
('TN = ', 7215102)
('FN = ', 567539)
('prec = ', 0.06702151018808485)
('test_accuracy = ', 0.46598565578460693)
('train_batches=', 

In [47]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 100000
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tnb = next_batch(Lab.shape[0], batch_size)
    vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
        print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
            
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            
            learn_step +=1  
        train_loss /= train_batches
        vali_loss = 0
        vali_accuracy = 0
        vali_batches = vx.shape[0]//batch_size
        print()
        
        print('vali_batches=',vali_batches)
        for j in range(vali_batches):
#             print('\rj: %d'%j)
            select = next(vnb)
            batch_X = vx[select]
            batch_Y  = vy[select]
            batch_X1 = vx1[select]
            batch_X=np.multiply(batch_X,1.0/mx)
            rev_batch_y=np.array(conv_mask_gt(batch_Y))

            vali_loss += sess.run(loss, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
            vali_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
        vali_loss /= vali_batches
        vali_accuracy /= vali_batches
        
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        print()
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TLoss:%.6f VLoss:%f VAcc:%f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, vali_loss, vali_accuracy))
        if learn_step > train_step:
                break
        print('learn_step=',learn_step)
        if learn_step % int(show_freq * train_batches)==0:
            
            
            TP = 0; FP = 0;TN = 0; FN = 0 
            #TP1=0;FP1=0
            num_images=batch_size
            n_chunks=np.shape(tx)[0]//batch_size
            tAcc=np.zeros(n_chunks)

            for chunk in range(0,n_chunks):               
                tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
                tx1_batch=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=conv_mask_gt(ty_batch)
                tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch, freqFeat: tx1_batch})
                a,b,c,d=compute_pos_neg(y2,p2)

                TP+=a; FP+=b;TN+=c; FN+=d
            
            prec=metrics(TP,FP,TN,FN)            
            test_accuracy=np.mean(tAcc)
       
            print(' | TP：%d FP:%d TN:%d FN:%d----prec:%.6f-----Tacc:%.6f' % (TP, FP, TN, FN, prec, test_accuracy))
            
            if prec > 0.30 :
                best_prec = prec
                save_path=saver.save(sess,'../model_s_gaijin1/final_model_nist.ckpt')
                print ("Best Model Found on NC16...")
                print ( "prec = "+str(prec) + ", acc = "+ str(test_accuracy))

            saver.save(sess, '../model_s_gaijin1/modelS%d.ckpt' % learn_step)
            
        
    saver.save(sess, '../model_s_gaijin1/modelS.ckpt')

('total_epoch=', 44.0)
('train_batches=', 2239)
2238()
('vali_batches=', 1)
()
05-22 19:36:51 | Step:  2239(  1/ 44) LR:0.000030 TLoss:0.640677 VLoss:0.619777 VAcc:0.630257
('learn_step=', 2239)
 | TP：435630 FP:4741920 TN:9642551 FN:908539----prec:0.084138-----Tacc:0.640754
('train_batches=', 2239)
2238()
('vali_batches=', 1)
()
05-22 19:57:30 | Step:  4478(  2/ 44) LR:0.000030 TLoss:0.599521 VLoss:0.604033 VAcc:0.704309
('learn_step=', 4478)
 | TP：365305 FP:2874600 TN:11509871 FN:978864----prec:0.112752-----Tacc:0.755005
('train_batches=', 2239)
2238()
('vali_batches=', 1)
()
05-22 20:17:39 | Step:  6717(  3/ 44) LR:0.000030 TLoss:0.587363 VLoss:0.600613 VAcc:0.740475
('learn_step=', 6717)
 | TP：120352 FP:961628 TN:13422843 FN:1223817----prec:0.111233-----Tacc:0.861054
('train_batches=', 2239)
2238()
('vali_batches=', 1)
()
05-22 20:37:57 | Step:  8956(  4/ 44) LR:0.000030 TLoss:0.579335 VLoss:0.587010 VAcc:0.788289
('learn_step=', 8956)
 | TP：50273 FP:406819 TN:13977652 FN:1293896---

 | TP：102378 FP:250686 TN:14133785 FN:1241791----prec:0.289970-----Tacc:0.905111
('train_batches=', 2239)
2238()
('vali_batches=', 1)
()
05-23 05:26:52 | Step: 67170( 30/ 44) LR:0.000030 TLoss:0.442226 VLoss:0.483823 VAcc:0.928511
('learn_step=', 67170)
 | TP：113299 FP:193323 TN:14191148 FN:1230870----prec:0.369507-----Tacc:0.909452
Best Model Found on NC16...
prec = 0.369507080379, acc = 0.9094522446393967
('train_batches=', 2239)
2238()
('vali_batches=', 1)
()
05-23 05:47:07 | Step: 69409( 31/ 44) LR:0.000030 TLoss:0.439532 VLoss:0.476985 VAcc:0.927225
('learn_step=', 69409)
 | TP：116520 FP:253567 TN:14130904 FN:1227649----prec:0.314845-----Tacc:0.905827
Best Model Found on NC16...
prec = 0.314844887824, acc = 0.9058268219232559
('train_batches=', 2239)
2238()
('vali_batches=', 1)
()
05-23 06:07:25 | Step: 71648( 32/ 44) LR:0.000030 TLoss:0.436895 VLoss:0.476548 VAcc:0.928157
('learn_step=', 71648)
 | TP：125797 FP:223103 TN:14161368 FN:1218372----prec:0.360553-----Tacc:0.908354
Best 

# model-train

In [ ]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 100000
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tnb = next_batch(Lab.shape[0], batch_size)
    vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_batches = int(Lab.shape[0] // batch_size)
        print('train_batches=',train_batches)
        
        for i in range(train_batches):
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = Img[select]
        
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            
            learn_step +=1  
        train_loss /= train_batches
        vali_loss = 0
        vali_accuracy = 0
        vali_batches = vx.shape[0]//batch_size
        print()
        
        print('vali_batches=',vali_batches)
        for j in range(vali_batches):
#             print('\rj: %d'%j,  end='', flush=True)
            select = next(vnb)
            batch_X = vx[select]
            batch_Y  = vy[select]
            batch_X1 = vx[select]
            batch_X=np.multiply(batch_X,1.0/mx)
            rev_batch_y=np.array(conv_mask_gt(batch_Y))

            vali_loss += sess.run(loss, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
            vali_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
        vali_loss /= vali_batches
        vali_accuracy /= vali_batches
        
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TLoss:%.6f VLoss:%f VAcc:%f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, vali_loss, vali_accuracy))
        
        
        if learn_step % train_batches==0:
            
            
            TP = 0; FP = 0;TN = 0; FN = 0 
            #TP1=0;FP1=0
            num_images=batch_size
            n_chunks=np.shape(tx)[0]//batch_size
            tAcc=np.zeros(n_chunks)

            for chunk in range(0,n_chunks):               
                tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
                tx1_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=conv_mask_gt(ty_batch)
                tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch, freqFeat: tx1_batch})
                a,b,c,d=compute_pos_neg(y2,p2)

                TP+=a; FP+=b;TN+=c; FN+=d
                 
            prec=metrics(TP,FP,TN,FN)            
            test_accuracy=np.mean(tAcc)

            print(' | TP：%d FP:%d TN:%d FN:%d----prec:%.6f-----Tacc:%.6f' % (TP, FP, TN, FN, prec, test_accuracy))
            
            if prec > 0.45 :
                best_prec = prec
                save_path=saver.save(sess,'../model_gai3/final_model_nist.ckpt')
                print ("Best Model Found on NC16...")
                print ( "prec = "+str(prec)+"("+str(best_prec)+")" + ", acc = "+ str(test_accuracy))

            saver.save(sess, '../model_gai3/modelS%d.ckpt' % learn_step)
            
        if learn_step > train_step:
                break
    saver.save(sess, '../model_gai3/modelS.ckpt')

('total_epoch=', 44.0)
('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-06 22:37:29 | Step:  2239(  1/ 44) LR:0.000030 TLoss:0.628372 VLoss:0.619009 VAcc:0.748359
 | TP：348867 FP:3787789 TN:10596682 FN:995302----prec:0.084336-----Tacc:0.695900
('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-06 23:03:00 | Step:  4478(  2/ 44) LR:0.000030 TLoss:0.598614 VLoss:0.605903 VAcc:0.909326
 | TP：144419 FP:1097300 TN:13287171 FN:1199750----prec:0.116306-----Tacc:0.853960
('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-06 23:28:21 | Step:  6717(  3/ 44) LR:0.000030 TLoss:0.588858 VLoss:0.598874 VAcc:0.928887
 | TP：107507 FP:531094 TN:13853377 FN:1236662----prec:0.168348-----Tacc:0.887610
('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-06 23:54:19 | Step:  8956(  4/ 44) LR:0.000030 TLoss:0.580286 VLoss:0.590473 VAcc:0.925045
 | TP：141039 FP:485421 TN:13899050 FN:1203130----prec:0.225136-----Tacc:0.892645
('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-07 00:20

('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-07 14:04:57 | Step: 82843( 37/ 44) LR:0.000030 TLoss:0.424850 VLoss:0.483029 VAcc:0.921375
 | TP：207146 FP:451539 TN:13932932 FN:1137023----prec:0.314484-----Tacc:0.899002
('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-07 14:30:49 | Step: 85082( 38/ 44) LR:0.000030 TLoss:0.423161 VLoss:0.477301 VAcc:0.926505
 | TP：199552 FP:449842 TN:13934629 FN:1144617----prec:0.307290-----Tacc:0.898627
('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-07 14:56:45 | Step: 87321( 39/ 44) LR:0.000030 TLoss:0.421657 VLoss:0.479705 VAcc:0.923668
 | TP：188483 FP:488914 TN:13895557 FN:1155686----prec:0.278246-----Tacc:0.895439
('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-07 15:22:49 | Step: 89560( 40/ 44) LR:0.000030 TLoss:0.420375 VLoss:0.479050 VAcc:0.922830
 | TP：180416 FP:457331 TN:13927140 FN:1163753----prec:0.282896-----Tacc:0.896934
('train_batches=', 2239)
2238()
('vali_batches=', 1)
06-07 15:47:57 | Step: 91799( 41/ 4

## PL-GNet

In [41]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 100000
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tnb = next_batch(Lab.shape[0], batch_size)
    vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
#         print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
#             print('batch_x1:',batch_x1.shape)
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            
            learn_step +=1  
        train_loss /= train_batches
        vali_loss = 0
        vali_accuracy = 0
        vali_batches = vx.shape[0]//batch_size
#         print()
        
#         print('vali_batches=',vali_batches)
        for j in range(vali_batches):
#             print('\rj: %d'%j)
            select = next(vnb)
            batch_X = vx[select]
            batch_Y  = vy[select]
            batch_X1 = vx1[select]
            batch_X=np.multiply(batch_X,1.0/mx)
            rev_batch_y=np.array(conv_mask_gt(batch_Y))

            vali_loss += sess.run(loss, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
            vali_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
        vali_loss /= vali_batches
        vali_accuracy /= vali_batches
        
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
       
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TLoss:%.6f VLoss:%f VAcc:%f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, vali_loss, vali_accuracy))
        if learn_step > train_step:
                break
        print('learn_step=',learn_step)
        if learn_step % int(show_freq * train_batches)==0:
            
            
            TP = 0; FP = 0;TN = 0; FN = 0 
            #TP1=0;FP1=0
            num_images=batch_size
            n_chunks=np.shape(tx)[0]//batch_size
            tAcc=np.zeros(n_chunks)

            for chunk in range(0,n_chunks):               
                tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
                tx1_batch=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=conv_mask_gt(ty_batch)
                tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch, freqFeat: tx1_batch})
                a,b,c,d=compute_pos_neg(y2,p2)

                TP+=a; FP+=b;TN+=c; FN+=d
            
            prec=metrics(TP,FP,TN,FN)            
            test_accuracy=np.mean(tAcc)
       
            print(' | TP：%d FP:%d TN:%d FN:%d----prec:%.6f-----Tacc:%.6f' % (TP, FP, TN, FN, prec, test_accuracy))
            
            if prec > 0.45 :
                best_prec = prec
                save_path=saver.save(sess,'../model_s_gaijin3/final_model_nist.ckpt')
                print ("Best Model Found on NC16...")
                print ( "prec = "+str(prec) + ", acc = "+ str(test_accuracy))

            saver.save(sess, '../model_s_gaijin3/modelS%d.ckpt' % learn_step)
            
        
    saver.save(sess, '../model_s_gaijin3/modelS.ckpt')

('total_epoch=', 44.0)
223806-10 15:26:20 | Step:  2239(  1/ 44) LR:0.000030 TLoss:0.628752 VLoss:0.625361 VAcc:0.681947
('learn_step=', 2239)
 | TP：480907 FP:3270213 TN:11114258 FN:863262----prec:0.128204-----Tacc:0.737201
223806-10 16:02:34 | Step:  4478(  2/ 44) LR:0.000030 TLoss:0.599538 VLoss:0.612402 VAcc:0.710164
('learn_step=', 4478)
 | TP：247650 FP:1389222 TN:12995249 FN:1096519----prec:0.151295-----Tacc:0.841962
223806-10 16:52:07 | Step:  6717(  3/ 44) LR:0.000030 TLoss:0.592080 VLoss:0.603283 VAcc:0.723325
('learn_step=', 6717)
 | TP：194531 FP:844816 TN:13539655 FN:1149638----prec:0.187167-----Tacc:0.873196
223806-10 17:44:08 | Step:  8956(  4/ 44) LR:0.000030 TLoss:0.585652 VLoss:0.592854 VAcc:0.874745
('learn_step=', 8956)
 | TP：162215 FP:571691 TN:13812780 FN:1181954----prec:0.221030-----Tacc:0.888506
223806-10 18:36:45 | Step: 11195(  5/ 44) LR:0.000030 TLoss:0.577894 VLoss:0.586371 VAcc:0.910894
('learn_step=', 11195)
 | TP：176918 FP:742802 TN:13641669 FN:1167251----pr

223806-12 01:35:52 | Step: 89560( 40/ 44) LR:0.000030 TLoss:0.421772 VLoss:0.466346 VAcc:0.928970
('learn_step=', 89560)
 | TP：209637 FP:360305 TN:14024166 FN:1134532----prec:0.367822-----Tacc:0.904961
223806-12 02:29:07 | Step: 91799( 41/ 44) LR:0.000030 TLoss:0.420680 VLoss:0.468464 VAcc:0.926008
('learn_step=', 91799)
 | TP：216008 FP:412336 TN:13972135 FN:1128161----prec:0.343773-----Tacc:0.902058
223806-12 03:22:08 | Step: 94038( 42/ 44) LR:0.000030 TLoss:0.419673 VLoss:0.469581 VAcc:0.927917
('learn_step=', 94038)
 | TP：215049 FP:431283 TN:13953188 FN:1129120----prec:0.332722-----Tacc:0.900792
223806-12 04:14:47 | Step: 96277( 43/ 44) LR:0.000030 TLoss:0.418850 VLoss:0.466866 VAcc:0.929807
('learn_step=', 96277)
 | TP：206728 FP:327885 TN:14056586 FN:1137441----prec:0.386687-----Tacc:0.906837
223806-12 05:08:13 | Step: 98516( 44/ 44) LR:0.000030 TLoss:0.418047 VLoss:0.469915 VAcc:0.929191
('learn_step=', 98516)
 | TP：201134 FP:354275 TN:14030196 FN:1143035----prec:0.362137-----Tacc

##  VGGAR_no_LSTM

In [ ]:
# batch_x=np.multiply(batch_x,1.0/mx)#####   测试集的数据忘记除以127了
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 1000000
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tnb = next_batch(Lab.shape[0], batch_size)
#     vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_accuracy = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
#         print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
            
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y})
            train_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_x, y: rev_batch_y})
            learn_step +=1  
        train_loss /= train_batches
        
        train_accuracy /= train_batches
#         vali_loss = 0
#         vali_accuracy = 0
#         vali_batches = vx.shape[0]//batch_size
        
        
#         print('vali_batches=',vali_batches)
#         for j in range(vali_batches):
# #             print('\rj: %d'%j)
#             select = next(vnb)
#             batch_X = vx[select]
#             batch_Y  = vy[select]
#             batch_X1 = vx1[select]
#             batch_X=np.multiply(batch_X,1.0/mx)
#             rev_batch_y=np.array(conv_mask_gt(batch_Y))

#             vali_loss += sess.run(loss, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
#             vali_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
#         vali_loss /= vali_batches
#         vali_accuracy /= vali_batches
        
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        print()
        
        if learn_step > train_step:
                break
        print('learn_step=',learn_step)
        
        
        
        if learn_step % int(show_freq * train_batches)==0:
            
            
            TP = 0; FP = 0;TN = 0; FN = 0 
            #TP1=0;FP1=0
            num_images=batch_size
            n_chunks=np.shape(tx)[0]//batch_size
            tAcc=np.zeros(n_chunks)
            vali_loss = 0
            for chunk in range(0,n_chunks):               
                tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                tx_batch=np.multiply(tx_batch,1.0/mx)
                ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
                tx1_batch=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=conv_mask_gt(ty_batch)
                
                vali_loss += sess.run(loss, feed_dict={input_layer: tx_batch, y:ty_batch})
        
                
                tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch})
                a,b,c,d=compute_pos_neg(y2,p2)

                TP+=a; FP+=b;TN+=c; FN+=d
            vali_loss /= n_chunks
            prec=metrics(TP,FP,TN,FN)            
            test_accuracy=np.mean(tAcc)
            print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TrainLoss:%.6f ValiLoss:%f TrainAcc:%.6f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, vali_loss,train_accuracy))

            print(' | TP：%d FP:%d TN:%d FN:%d----prec:%.6f-----Tacc:%.6f' % (TP, FP, TN, FN, prec, test_accuracy))
            
            if prec > 0.55 :
                best_prec = prec
                save_path=saver.save(sess,'../model_s_gaijin1/final_model_nist.ckpt')
                print ("Best Model Found on NC16...")
                print ( "prec = "+str(prec) + ", acc = "+ str(test_accuracy))

            saver.save(sess, '../model_s_gaijin1/modelS%d.ckpt' % learn_step)
            
        
    saver.save(sess, '../model_s_gaijin1/modelS.ckpt')

total_epoch= 497
2014
learn_step= 2015
07-25 15:58:28 | Step:  2015(  1/497) LR:0.000030 TrainLoss:0.616731 ValiLoss:0.597109 TrainAcc:0.746071
 | TP：14925079 FP:59139493 TN:360222831 FN:4148437----prec:0.201514-----Tacc:0.855654
2014
learn_step= 4030
07-25 16:34:50 | Step:  4030(  2/497) LR:0.000030 TrainLoss:0.595716 ValiLoss:0.588660 TrainAcc:0.895656
 | TP：10989560 FP:15379091 TN:403983233 FN:8083956----prec:0.416766-----Tacc:0.946485
2014
learn_step= 6045
07-25 17:11:29 | Step:  6045(  3/497) LR:0.000030 TrainLoss:0.588066 ValiLoss:0.581384 TrainAcc:0.949847
 | TP：10716981 FP:10080495 TN:409281829 FN:8356535----prec:0.515302-----Tacc:0.957949
2014
learn_step= 8060
07-25 17:49:22 | Step:  8060(  4/497) LR:0.000030 TrainLoss:0.581102 ValiLoss:0.574486 TrainAcc:0.960646
 | TP：11008309 FP:8590087 TN:410772237 FN:8065207----prec:0.561694-----Tacc:0.962012
Best Model Found on NC16...
prec = 0.5616943856017576, acc = 0.9620122064924026
2014
learn_step= 10075
07-25 18:30:13 | Step: 10075(

Best Model Found on NC16...
prec = 0.8053540037937877, acc = 0.9799020886421204
2014
learn_step= 58435
07-26 16:00:07 | Step: 58435( 29/497) LR:0.000030 TrainLoss:0.448730 ValiLoss:0.431149 TrainAcc:0.984842
 | TP：13418484 FP:3183421 TN:416178903 FN:5655032----prec:0.808250-----Tacc:0.979841
Best Model Found on NC16...
prec = 0.8082496556870985, acc = 0.9798409524519882
2014
learn_step= 60450
07-26 16:55:07 | Step: 60450( 30/497) LR:0.000030 TrainLoss:0.445583 ValiLoss:0.427691 TrainAcc:0.985026
 | TP：13612162 FP:3331992 TN:416030332 FN:5461354----prec:0.803354-----Tacc:0.979944
Best Model Found on NC16...
prec = 0.8033544784826625, acc = 0.9799438281978727
2014
learn_step= 62465
07-26 17:50:13 | Step: 62465( 31/497) LR:0.000030 TrainLoss:0.442603 ValiLoss:0.424505 TrainAcc:0.985169
 | TP：13813421 FP:3367539 TN:415994785 FN:5260095----prec:0.803996-----Tacc:0.980322
Best Model Found on NC16...
prec = 0.8039958768310921, acc = 0.9803217873979577
2014
learn_step= 64480
07-26 18:45:43 | S

## NIST2016

In [21]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 3000
batch_size = 30

Img_NC16 = np.load('../dataset_npy/NC_16/train/NC16_train_img.npy')
Img_NC16_img=np.shape(Img_NC16)
print ('Img_NC16_img.shape=',Img_NC16_img)
Lab_NC16 = np.load('../dataset_npy/NC_16/train/NC16_train_label.npy')
# feat_nc16 = np.load('../dataset_npy/NC_16/train/NC16_train_img_feat.npy')
feat_nc16 = np.load('../dataset_npy/NC_16/train/NC16_train_img_cohere_feat.npy')

vx=Img_NC16[-40:]
vx1=feat_nc16[-40:]
vy=Lab_NC16[-40:]


Img = Img_NC16[:-40]
Lab = Lab_NC16[:-40]
freq1 = feat_nc16[:-40]
print('Img_NC16.shape=',Img_NC16.shape)




with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    saver.restore(sess,'../model_s_gaijin4/modelS251875.ckpt')
    print ('session starting .................!!!!')
    
    tnb = next_batch(Lab.shape[0], batch_size)
#     vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_accuracy = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
#         print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
            
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            learn_step +=1  
        train_loss /= train_batches
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        print()
        if learn_step > train_step:
                break
        train_accuracy /= train_batches
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TrainLoss:%.6f TrainAcc:%.6f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, train_accuracy))
       

            
        
    saver.save(sess, '../model_s/modelS.ckpt')

Img_NC16_img.shape= (798, 256, 256, 3)
Img_NC16.shape= (798, 256, 256, 3)
INFO:tensorflow:Restoring parameters from ../model_s_gaijin4/modelS251875.ckpt
session starting .................!!!!
total_epoch= 120
24
07-03 17:14:35 | Step:    25(  1/120) LR:0.000030 TrainLoss:0.443959 TrainAcc:0.933720
24
07-03 17:14:51 | Step:    50(  2/120) LR:0.000030 TrainLoss:0.424094 TrainAcc:0.938031
24
07-03 17:15:06 | Step:    75(  3/120) LR:0.000030 TrainLoss:0.415357 TrainAcc:0.943204
24
07-03 17:15:22 | Step:   100(  4/120) LR:0.000030 TrainLoss:0.410943 TrainAcc:0.946128
24
07-03 17:15:38 | Step:   125(  5/120) LR:0.000030 TrainLoss:0.405904 TrainAcc:0.949611
24
07-03 17:15:53 | Step:   150(  6/120) LR:0.000030 TrainLoss:0.401995 TrainAcc:0.955328
24
07-03 17:16:09 | Step:   175(  7/120) LR:0.000030 TrainLoss:0.397012 TrainAcc:0.961794
24
07-03 17:16:25 | Step:   200(  8/120) LR:0.000030 TrainLoss:0.395596 TrainAcc:0.964764
24
07-03 17:16:41 | Step:   225(  9/120) LR:0.000030 TrainLoss:0.393501

24
07-03 17:38:05 | Step:  2250( 90/120) LR:0.000030 TrainLoss:0.374835 TrainAcc:0.991106
24
07-03 17:38:21 | Step:  2275( 91/120) LR:0.000030 TrainLoss:0.375841 TrainAcc:0.990501
24
07-03 17:38:37 | Step:  2300( 92/120) LR:0.000030 TrainLoss:0.376310 TrainAcc:0.990176
24
07-03 17:38:52 | Step:  2325( 93/120) LR:0.000030 TrainLoss:0.376115 TrainAcc:0.990056
24
07-03 17:39:08 | Step:  2350( 94/120) LR:0.000030 TrainLoss:0.375145 TrainAcc:0.990749
24
07-03 17:39:24 | Step:  2375( 95/120) LR:0.000030 TrainLoss:0.374466 TrainAcc:0.991417
24
07-03 17:39:40 | Step:  2400( 96/120) LR:0.000030 TrainLoss:0.374602 TrainAcc:0.991308
24
07-03 17:39:56 | Step:  2425( 97/120) LR:0.000030 TrainLoss:0.375582 TrainAcc:0.990724
24
07-03 17:40:12 | Step:  2450( 98/120) LR:0.000030 TrainLoss:0.375103 TrainAcc:0.990885
24
07-03 17:40:28 | Step:  2475( 99/120) LR:0.000030 TrainLoss:0.374799 TrainAcc:0.991201
24
07-03 17:40:44 | Step:  2500(100/120) LR:0.000030 TrainLoss:0.376141 TrainAcc:0.990201
24
07-03 1

In [27]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 3000
batch_size = 30

Img = np.load('../dataset_npy/NC_16/mani/NC16_mani_train_img.npy')
Lab = np.load('../dataset_npy/NC_16/mani/NC16_mani_train_label.npy')
freq1 = np.load('../dataset_npy/NC_16/mani/NC16_mani_train_imgS_feat.npy')
print('Img_NC16.shape=',Img_NC16.shape)

with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    saver.restore(sess,'../model_s_gaijin4/modelS251875.ckpt')
    print ('session starting .................!!!!')
    
    tnb = next_batch(Lab.shape[0], batch_size)
#     vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_accuracy = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
#         print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
            
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            learn_step +=1  
        train_loss /= train_batches
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        print()
        if learn_step > train_step:
                break
        train_accuracy /= train_batches
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TrainLoss:%.6f TrainAcc:%.6f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, train_accuracy))
       

            
        
    saver.save(sess, '../model_s_gaijin6/modelS.ckpt')

Img_NC16.shape= (798, 256, 256, 3)
INFO:tensorflow:Restoring parameters from ../model_s_gaijin4/modelS251875.ckpt
session starting .................!!!!
total_epoch= 231
12
07-05 10:09:51 | Step:    13(  1/231) LR:0.000030 TrainLoss:0.458275 TrainAcc:0.932608
12
07-05 10:09:58 | Step:    26(  2/231) LR:0.000030 TrainLoss:0.438413 TrainAcc:0.937123
12
07-05 10:10:05 | Step:    39(  3/231) LR:0.000030 TrainLoss:0.429661 TrainAcc:0.937196
12
07-05 10:10:12 | Step:    52(  4/231) LR:0.000030 TrainLoss:0.419716 TrainAcc:0.941483
12
07-05 10:10:20 | Step:    65(  5/231) LR:0.000030 TrainLoss:0.418053 TrainAcc:0.943267
12
07-05 10:10:27 | Step:    78(  6/231) LR:0.000030 TrainLoss:0.413329 TrainAcc:0.946106
12
07-05 10:10:34 | Step:    91(  7/231) LR:0.000030 TrainLoss:0.411844 TrainAcc:0.949005
12
07-05 10:10:41 | Step:   104(  8/231) LR:0.000030 TrainLoss:0.409231 TrainAcc:0.951242
12
07-05 10:10:49 | Step:   117(  9/231) LR:0.000030 TrainLoss:0.406971 TrainAcc:0.954734
12
07-05 10:10:56 | 

12
07-05 10:21:08 | Step:  1183( 91/231) LR:0.000030 TrainLoss:0.377427 TrainAcc:0.988920
12
07-05 10:21:15 | Step:  1196( 92/231) LR:0.000030 TrainLoss:0.377601 TrainAcc:0.988791
12
07-05 10:21:23 | Step:  1209( 93/231) LR:0.000030 TrainLoss:0.378637 TrainAcc:0.988041
12
07-05 10:21:31 | Step:  1222( 94/231) LR:0.000030 TrainLoss:0.378776 TrainAcc:0.988278
12
07-05 10:21:38 | Step:  1235( 95/231) LR:0.000030 TrainLoss:0.378810 TrainAcc:0.987615
12
07-05 10:21:46 | Step:  1248( 96/231) LR:0.000030 TrainLoss:0.376422 TrainAcc:0.989859
12
07-05 10:21:54 | Step:  1261( 97/231) LR:0.000030 TrainLoss:0.377576 TrainAcc:0.988787
12
07-05 10:22:01 | Step:  1274( 98/231) LR:0.000030 TrainLoss:0.377328 TrainAcc:0.988750
12
07-05 10:22:09 | Step:  1287( 99/231) LR:0.000030 TrainLoss:0.377773 TrainAcc:0.988791
12
07-05 10:22:16 | Step:  1300(100/231) LR:0.000030 TrainLoss:0.376130 TrainAcc:0.990189
12
07-05 10:22:24 | Step:  1313(101/231) LR:0.000030 TrainLoss:0.376031 TrainAcc:0.990379
12
07-05 1

12
07-05 10:32:45 | Step:  2379(183/231) LR:0.000030 TrainLoss:0.375250 TrainAcc:0.991083
12
07-05 10:32:53 | Step:  2392(184/231) LR:0.000030 TrainLoss:0.376472 TrainAcc:0.990146
12
07-05 10:33:01 | Step:  2405(185/231) LR:0.000030 TrainLoss:0.374889 TrainAcc:0.991322
12
07-05 10:33:08 | Step:  2418(186/231) LR:0.000030 TrainLoss:0.376530 TrainAcc:0.989685
12
07-05 10:33:16 | Step:  2431(187/231) LR:0.000030 TrainLoss:0.375359 TrainAcc:0.991005
12
07-05 10:33:23 | Step:  2444(188/231) LR:0.000030 TrainLoss:0.373830 TrainAcc:0.992353
12
07-05 10:33:31 | Step:  2457(189/231) LR:0.000030 TrainLoss:0.375129 TrainAcc:0.991189
12
07-05 10:33:38 | Step:  2470(190/231) LR:0.000030 TrainLoss:0.374455 TrainAcc:0.991667
12
07-05 10:33:46 | Step:  2483(191/231) LR:0.000030 TrainLoss:0.375189 TrainAcc:0.991273
12
07-05 10:33:54 | Step:  2496(192/231) LR:0.000030 TrainLoss:0.375265 TrainAcc:0.991295
12
07-05 10:34:01 | Step:  2509(193/231) LR:0.000030 TrainLoss:0.374105 TrainAcc:0.992078
12
07-05 1

In [88]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 3000
batch_size = 30


Img = np.load('../dataset_npy/IFS/IFS_imgS.npy')
Lab = np.load('../dataset_npy/IFS/IFS_labelS.npy')
freq1 = np.load('../dataset_npy/IFS/IFS_img_feat.npy')
print('Img.shape=',Img.shape)
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    saver.restore(sess,'../model_s_gaijin4/modelS251875.ckpt')
    print ('session starting .................!!!!')
    
    tnb = next_batch(Lab.shape[0], batch_size)
#     vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_accuracy = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
#         print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
           
            rev_batch_y=np.array(conv_mask_gt(batch_y))
#             print('batch_y',rev_batch_y.max())
#             print('batch_y',rev_batch_y.min())
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            learn_step +=1  
        train_loss /= train_batches
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        print()
        if learn_step > train_step:
                break
        train_accuracy /= train_batches
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TrainLoss:%.6f TrainAcc:%.6f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, train_accuracy))
       

            
        
    saver.save(sess, '../model_finetune_IFC/modelS.ckpt')

Img.shape= (441, 256, 256, 3)
INFO:tensorflow:Restoring parameters from ../model_s_gaijin4/modelS251875.ckpt
session starting .................!!!!
total_epoch= 215
13
07-27 17:57:08 | Step:    14(  1/215) LR:0.000030 TrainLoss:0.454782 TrainAcc:0.947981
13
07-27 17:57:17 | Step:    28(  2/215) LR:0.000030 TrainLoss:0.436918 TrainAcc:0.953111
13
07-27 17:57:26 | Step:    42(  3/215) LR:0.000030 TrainLoss:0.425745 TrainAcc:0.954204
13
07-27 17:57:34 | Step:    56(  4/215) LR:0.000030 TrainLoss:0.418862 TrainAcc:0.954022
13
07-27 17:57:43 | Step:    70(  5/215) LR:0.000030 TrainLoss:0.413757 TrainAcc:0.955117
13
07-27 17:57:51 | Step:    84(  6/215) LR:0.000030 TrainLoss:0.410306 TrainAcc:0.956543
13
07-27 17:58:00 | Step:    98(  7/215) LR:0.000030 TrainLoss:0.407649 TrainAcc:0.957021
13
07-27 17:58:08 | Step:   112(  8/215) LR:0.000030 TrainLoss:0.405518 TrainAcc:0.957812
13
07-27 17:58:16 | Step:   126(  9/215) LR:0.000030 TrainLoss:0.403720 TrainAcc:0.957918
13
07-27 17:58:25 | Step:

13
07-27 18:10:11 | Step:  1274( 91/215) LR:0.000030 TrainLoss:0.380245 TrainAcc:0.984665
13
07-27 18:10:19 | Step:  1288( 92/215) LR:0.000030 TrainLoss:0.380913 TrainAcc:0.984036
13
07-27 18:10:28 | Step:  1302( 93/215) LR:0.000030 TrainLoss:0.380688 TrainAcc:0.984328
13
07-27 18:10:37 | Step:  1316( 94/215) LR:0.000030 TrainLoss:0.380671 TrainAcc:0.984402
13
07-27 18:10:46 | Step:  1330( 95/215) LR:0.000030 TrainLoss:0.380653 TrainAcc:0.984204
13
07-27 18:10:54 | Step:  1344( 96/215) LR:0.000030 TrainLoss:0.380804 TrainAcc:0.984095
13
07-27 18:11:03 | Step:  1358( 97/215) LR:0.000030 TrainLoss:0.380273 TrainAcc:0.984641
13
07-27 18:11:11 | Step:  1372( 98/215) LR:0.000030 TrainLoss:0.380330 TrainAcc:0.984601
13
07-27 18:11:20 | Step:  1386( 99/215) LR:0.000030 TrainLoss:0.380645 TrainAcc:0.984157
13
07-27 18:11:29 | Step:  1400(100/215) LR:0.000030 TrainLoss:0.380462 TrainAcc:0.984624
13
07-27 18:11:37 | Step:  1414(101/215) LR:0.000030 TrainLoss:0.380525 TrainAcc:0.984294
13
07-27 1

13
07-27 18:23:30 | Step:  2562(183/215) LR:0.000030 TrainLoss:0.377897 TrainAcc:0.986012
13
07-27 18:23:38 | Step:  2576(184/215) LR:0.000030 TrainLoss:0.377937 TrainAcc:0.986031
13
07-27 18:23:47 | Step:  2590(185/215) LR:0.000030 TrainLoss:0.378347 TrainAcc:0.985452
13
07-27 18:23:56 | Step:  2604(186/215) LR:0.000030 TrainLoss:0.377605 TrainAcc:0.986409
13
07-27 18:24:04 | Step:  2618(187/215) LR:0.000030 TrainLoss:0.378180 TrainAcc:0.985594
13
07-27 18:24:13 | Step:  2632(188/215) LR:0.000030 TrainLoss:0.378662 TrainAcc:0.985345
13
07-27 18:24:22 | Step:  2646(189/215) LR:0.000030 TrainLoss:0.378212 TrainAcc:0.985534
13
07-27 18:24:30 | Step:  2660(190/215) LR:0.000030 TrainLoss:0.377815 TrainAcc:0.985933
13
07-27 18:24:39 | Step:  2674(191/215) LR:0.000030 TrainLoss:0.378072 TrainAcc:0.985749
13
07-27 18:24:48 | Step:  2688(192/215) LR:0.000030 TrainLoss:0.377873 TrainAcc:0.985834
13
07-27 18:24:56 | Step:  2702(193/215) LR:0.000030 TrainLoss:0.377792 TrainAcc:0.985973
13
07-27 1

## IFC

In [ ]:
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
#     saver.restore(sess,'../model_s_gaijin6/modelS.ckpt')
    saver.restore(sess,'../model_finetune_IFC/modelS.ckpt')
    print ('session starting .................!!!!')
            
    TP = 0; FP = 0;TN = 0; FN = 0 
    #TP1=0;FP1=0
    num_images=batch_size
    
    tx = np.load('../dataset_npy/IFS/IFS_imgS.npy')
    tx= np.multiply(tx,1.0/mx)
    ty = np.load('../dataset_npy/IFS/IFS_labelS.npy')
    freq4 = np.load('../dataset_npy/IFS/IFS_img_feat.npy')
    
    print('ty.shape:',ty.shape)
    nTx=np.zeros((batch_size,256,256,3))
    nTy=np.zeros((batch_size,256,256))
#     nTx1=np.zeros((batch_size,64,240))
    nTx1=np.zeros((batch_size,248,248,3))
    
    n_chunks=np.shape(tx)[0]//batch_size
    tAcc=np.zeros(n_chunks)
    
    n1=0;n2=len(tx)
    pred = np.zeros([441,256,256])
    prob = np.zeros([256*256*441,2])
    for chunk in range(n1,n_chunks):
#         nTx[imNb-n1]=tx[imNb] 
#         nTy[imNb-n1]=ty[imNb]
#         nTx1[imNb-n1]=freq4[imNb]
        print('chunk=',chunk)
        nTx=tx[((chunk)*num_images):((chunk+1)*num_images),...]
        
        nTy=ty[((chunk)*num_images):((chunk+1)*num_images),...]
#         print(nTy.shape)
        nTx1=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
       
        
#         print ('nTx = ',np.shape(nTx))
#         print ('nTy = ',np.shape(nTy))
#         print ('nTx1 = ',np.shape(nTx1))
        ty_prime=np.array(conv_mask_gt(nTy))
            
        final_predictions, final_probabilities,y2=sess.run([mask_pred,probabilities,mask_actual], feed_dict={input_layer: nTx, y:ty_prime, freqFeat: nTx1})
    
        print ('final_predictions_shape:',np.shape(final_predictions))
        print ('final_probabilities_shape;',np.shape(final_probabilities))
        pred[((chunk)*num_images):((chunk+1)*num_images),...] =  final_predictions
        prob[((chunk)*num_images)*256*256:((chunk+1)*num_images)*256*256,...] = final_probabilities
        
        print ('pred_shape:',np.shape(pred))
        print ('prob_shape:',np.shape(prob))
    print('-----------------------------------------------------------')
    print ('pred_shape:',np.shape(pred))
    print ('prob_shape:',np.shape(prob))
    final_predictions1 =pred 
    final_probabilities1 =prob
    print ('final_predictions1_shape:',np.shape(final_predictions1))
    print ('final_probabilities1_shape;',np.shape(final_probabilities1))
    
    #sio.savemat('pred_res.mat',{'img':nTx,'labels':nTy,'pred':final_predictions,'prob':final_probabilities,'gT':y2})
    nb = 0
    F1_sum =0
#     for i in range(n1,n2):
    for i in range(n1,441):
        print('i=',i)
        cmap = plt.get_cmap('bwr')
        f,(ax,ax1,ax2)=plt.subplots(1,3,sharey=True)
#         f,(ax,ax1,ax2,ax3)=plt.subplots(1,4,sharey=True)
        ax.imshow(tx[i])
        ax1.imshow(ty[i])
        ax2.imshow(final_predictions1[i])
        plt.imsave(fname='/home/shizenan/IFS_result/tx_%s.png'%i,
                       arr=tx[i], cmap=plt.cm.gray)
        plt.imsave(fname='/home/shizenan/IFS_result/ty_%s.png'%i,
                       arr=ty[i], cmap=plt.cm.gray)
        plt.imsave(fname='/home/shizenan/IFS_result/final_predictions_%s.png'%i,
                       arr=final_predictions1[i], cmap=plt.cm.gray)
        F1 = F1_score(final_predictions1[i],ty[i])
        
        
        #ax3.set_title('Final Argmax')
#         probability_graph = ax3.imshow(final_probabilities.squeeze()[nb, :,:, 0])
        nb += 1
        #ax3.set_title('Final Probability of the Class')
#         plt.colorbar(probability_graph)
        plt.show()
        print('F1:,i',F1,i)
        F1_sum += F1
    avg_F1 = F1_sum/(n2-n1)
    print('F1_sum:',F1_sum)
    print('avg_F1:',avg_F1)
    print('ty.shape:',ty.shape)
    print('final_predictions1.shape:',final_predictions1.shape)
    print('final_probabilities1.shape:',final_probabilities1.shape)
    print('-----------------------------------------------------------')
    
    prob_new1=np.zeros(len(final_probabilities1))
# prob_new1 = prob_new[:,0]

# prob_new=np.zeros(len(final_probabilities1))
#     for i in range(0,150):
#         if final_probabilities1[i,0]>= final_probabilities1[i,1]:
#             prob_new[i]= final_probabilities1[i,0]
#         if final_probabilities1[i,0]<= final_probabilities1[i,1]:
#               prob_new[i]= final_probabilities1[i,1]
    ty = ty[0:150,]
    ty= ty.flatten()
    for i in range(0,len(final_probabilities1)):
        if ty[i]==0.:
            prob_new1[i]= final_probabilities1[i,1]
        if ty[i]==1.:
            prob_new1[i]= final_probabilities1[i,0]
            
    print(prob_new1.shape)
    print(prob_new1.shape)


    ccc= check_fit(ty, prob_new1)

## Test NIST

In [29]:
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    saver.restore(sess,'../model_s_gaijin6/modelS.ckpt')
    print ('session starting .................!!!!')
            
    TP = 0; FP = 0;TN = 0; FN = 0 
    #TP1=0;FP1=0
    num_images=batch_size
    
    tx = np.load('../dataset_npy/NC_16/mani/NC16_mani_test_img.npy')
    ty = np.load('../dataset_npy/NC_16/mani/NC16_mani_test_label.npy')
    freq4 = np.load('../dataset_npy/NC_16/mani/NC16_mani_test_imgS_feat.npy')
    print(ty.shape)
    
    
    n_chunks=np.shape(tx)[0]//batch_size
    tAcc=np.zeros(n_chunks)
    vali_loss = 0
    for chunk in range(0,n_chunks):
        
        sys.stdout.write('\r' + str(chunk))
        sys.stdout.flush()
        
        tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
        tx_batch=np.multiply(tx_batch,1.0/mx)
        ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
        tx1_batch=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
        ty_batch=conv_mask_gt(ty_batch)

        vali_loss += sess.run(loss, feed_dict={input_layer: tx_batch, y:ty_batch, freqFeat: tx1_batch})


        tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch, freqFeat: tx1_batch})
        a,b,c,d=compute_pos_neg(y2,p2)

        TP+=a; FP+=b;TN+=c; FN+=d
    vali_loss /= n_chunks
    prec=metrics(TP,FP,TN,FN)            
    test_accuracy=np.mean(tAcc)

    print(' | TP：%d FP:%d TN:%d FN:%d----prec:%.6f-----Tacc:%.6f' % (TP, FP, TN, FN, prec, test_accuracy))

    if prec > 0.70 :
        best_prec = prec
        print ("Best Model Found on NC16...")
        print ( "prec = "+str(prec) + ", acc = "+ str(test_accuracy))


INFO:tensorflow:Restoring parameters from ../model_s_gaijin6/modelS.ckpt
session starting .................!!!!
(158, 256, 256)
4 | TP：560038 FP:43299 TN:9034868 FN:192195----prec:0.928234-----Tacc:0.976044
Best Model Found on NC16...
prec = 0.9282341378034286, acc = 0.9760443091392517


## Test F1  NC16

In [ ]:
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    saver.restore(sess,'../model_s_gaijin6/modelS.ckpt')
    print ('session starting .................!!!!')
            
    TP = 0; FP = 0;TN = 0; FN = 0 
    #TP1=0;FP1=0
    num_images=batch_size
    
#     tx = np.load('../dataset_npy/NC_16/NC16_mani_imgS.npy')
#     tx= np.multiply(tx,1.0/mx)
#     ty = np.load('../dataset_npy/NC_16/NC16_mani_labelS.npy')
#     freq4 = np.load('../dataset_npy/NC_16/NC16_mani_imgS_feat.npy')
    tx = np.load('../dataset_npy/NC_16/mani/NC16_mani_test_img.npy')
    tx= np.multiply(tx,1.0/mx)
    ty = np.load('../dataset_npy/NC_16/mani/NC16_mani_test_label.npy')
    freq4 = np.load('../dataset_npy/NC_16/mani/NC16_mani_test_imgS_feat.npy')
    
    
    print('ty.shape:',ty.shape)
    nTx=np.zeros((batch_size,256,256,3))
    nTy=np.zeros((batch_size,256,256))
#     nTx1=np.zeros((batch_size,64,240))
    nTx1=np.zeros((batch_size,248,248,3))
    
    n_chunks=np.shape(tx)[0]//batch_size
    tAcc=np.zeros(n_chunks)
    
    n1=0;n2=len(tx)
    pred = np.zeros([150,256,256])
    prob = np.zeros([256*256*150,2])
    for chunk in range(n1,n_chunks):
#         nTx[imNb-n1]=tx[imNb] 
#         nTy[imNb-n1]=ty[imNb]
#         nTx1[imNb-n1]=freq4[imNb]
        print('chunk=',chunk)
        nTx=tx[((chunk)*num_images):((chunk+1)*num_images),...]
        
        nTy=ty[((chunk)*num_images):((chunk+1)*num_images),...]
#         print(nTy.shape)
        nTx1=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
       
        
#         print ('nTx = ',np.shape(nTx))
#         print ('nTy = ',np.shape(nTy))
#         print ('nTx1 = ',np.shape(nTx1))
        ty_prime=conv_mask_gt(nTy)
        final_predictions, final_probabilities,y2=sess.run([mask_pred,probabilities,mask_actual], feed_dict={input_layer: nTx, y:ty_prime, freqFeat: nTx1})
    
        print ('final_predictions_shape:',np.shape(final_predictions))
        print ('final_probabilities_shape;',np.shape(final_probabilities))
        pred[((chunk)*num_images):((chunk+1)*num_images),...] =  final_predictions
        prob[((chunk)*num_images)*256*256:((chunk+1)*num_images)*256*256,...] = final_probabilities
        
        print ('pred_shape:',np.shape(pred))
        print ('prob_shape:',np.shape(prob))
    print('-----------------------------------------------------------')
    print ('pred_shape:',np.shape(pred))
    print ('prob_shape:',np.shape(prob))
    final_predictions1 =pred 
    final_probabilities1 =prob
    print ('final_predictions1_shape:',np.shape(final_predictions1))
    print ('final_probabilities1_shape;',np.shape(final_probabilities1))
    
    #sio.savemat('pred_res.mat',{'img':nTx,'labels':nTy,'pred':final_predictions,'prob':final_probabilities,'gT':y2})
    nb = 0
    F1_sum =0
#     for i in range(n1,n2):
    for i in range(n1,150):
        print('i=',i)
        cmap = plt.get_cmap('bwr')
        f,(ax,ax1,ax2)=plt.subplots(1,3,sharey=True)
#         f,(ax,ax1,ax2,ax3)=plt.subplots(1,4,sharey=True)
        ax.imshow(tx[i])
        ax1.imshow(ty[i])
        ax2.imshow(final_predictions1[i])
        plt.imsave(fname='/home/shizenan/shi_tmp/tx_%s.png'%i,
                       arr=tx[i], cmap=plt.cm.gray)
        plt.imsave(fname='/home/shizenan/shi_tmp/ty_%s.png'%i,
                       arr=ty[i], cmap=plt.cm.gray)
        plt.imsave(fname='/home/shizenan/shi_tmp/final_predictions_%s.png'%i,
                       arr=final_predictions1[i], cmap=plt.cm.gray)
        F1 = F1_score(final_predictions1[i],ty[i])
        
        
        #ax3.set_title('Final Argmax')
#         probability_graph = ax3.imshow(final_probabilities.squeeze()[nb, :,:, 0])
        nb += 1
        #ax3.set_title('Final Probability of the Class')
#         plt.colorbar(probability_graph)
        plt.show()
        print('F1:,i',F1,i)
        F1_sum += F1
    avg_F1 = F1_sum/(n2-n1)
    print('F1_sum:',F1_sum)
    print('avg_F1:',avg_F1)
    print('ty.shape:',ty.shape)
    print('final_predictions1.shape:',final_predictions1.shape)
    print('final_probabilities1.shape:',final_probabilities1.shape)
    print('-----------------------------------------------------------')
    
    prob_new1=np.zeros(len(final_probabilities1))
# prob_new1 = prob_new[:,0]

# prob_new=np.zeros(len(final_probabilities1))
#     for i in range(0,150):
#         if final_probabilities1[i,0]>= final_probabilities1[i,1]:
#             prob_new[i]= final_probabilities1[i,0]
#         if final_probabilities1[i,0]<= final_probabilities1[i,1]:
#               prob_new[i]= final_probabilities1[i,1]
    ty = ty[0:150,]
    ty= ty.flatten()
    for i in range(0,len(final_probabilities1)):
        if ty[i]==0.:
            prob_new1[i]= final_probabilities1[i,1]
        if ty[i]==1.:
            prob_new1[i]= final_probabilities1[i,0]
            
    print(prob_new1.shape)
    print(prob_new1.shape)


    ccc= check_fit(ty, prob_new1)